In [10]:
from tropy.learn import fit_classifier, _inrad_eigenpair, fit_classifier_onevsall, predict_onevsall
from tropy.metrics import accuracy_multiple, veronese_feature_names, print_features_per_class
from tropy.ops import veronese
from tropy.utils import simplex_lattice_points
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=3, suppress=True)

In [11]:
base_df = pd.read_csv('./data/players_22.csv')
df = base_df.loc[:, 'attacking_crossing':'goalkeeping_reflexes']
classes = ["ST", "CB", "CM", "GK"]

lattice_points = None
def class_df(class_name, size=None):
  global lattice_points
  df_class = df[base_df["player_positions"].str.contains(class_name)]
  df_train, df_test = train_test_split(df_class, test_size=0.2, random_state=43)
  Ctrain, Ctest = df_train.to_numpy(dtype=float).T, df_test.to_numpy(dtype=float).T

  print(Ctrain.shape, Ctest.shape)
  if size is not None:
    d = Ctrain.shape[0]
    lattice_points = list(simplex_lattice_points(d, size))
    Ctrain, Ctest = veronese(lattice_points, Ctrain), veronese(lattice_points, Ctest)
  return Ctrain, Ctest

/var/folders/4k/mk0hky353gj5hxw05rvj1c_00000gn/T/ipykernel_25368/3403427926.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv('./data/players_22.csv')


In [12]:
size = 2

In [13]:
Clist_train, Clist_test = [], []
for class_name in classes:
  train, test = class_df(class_name, size)
  Clist_train.append(train)
  Clist_test.append(test)

(34, 2718) (34, 680)
(34, 3216) (34, 805)
(34, 3363) (34, 841)
(34, 1705) (34, 427)


In [14]:
x, l = _inrad_eigenpair(Clist_train, N=50, old_method=True)
print("Apex:", x)
print("Eigval:", l)

 82%|████████▏ | 41/50 [00:11<00:02,  3.47it/s]

Apex: [-30.279 -30.279 -12.279 ... -35.279 -36.279 -37.279]
Eigval: -70.99999999993952


In [15]:
predictor, sector_indicator = fit_classifier(Clist_train, x)

In [16]:
accuracy = accuracy_multiple(predictor, Clist_test)
print(f"Accuracy: {round(accuracy, 3)}")

Accuracy: 0.784


In [17]:
indicators, apices = fit_classifier_onevsall(Clist_train)
accuracy = accuracy_multiple(predict_onevsall(indicators, apices, Clist_train), Clist_test)
print(f"Accuracy (one vs. all): {round(accuracy, 3)}")

 78%|███████▊  | 39/50 [00:10<00:03,  3.65it/s]


Accuracy (one vs. all): 0.761


In [18]:
print_features_per_class(classes, veronese_feature_names(df.columns.to_list(), lattice_points), sector_indicator)

Dominant features for each class:
- ST: attacking_finishing + attacking_heading_accuracy, attacking_finishing + attacking_volleys, attacking_finishing + skill_dribbling, attacking_finishing + skill_fk_accuracy, attacking_finishing + movement_balance, attacking_finishing + power_long_shots, attacking_finishing + mentality_aggression, attacking_finishing + mentality_positioning, attacking_finishing + mentality_penalties, attacking_heading_accuracy + mentality_positioning, attacking_volleys + mentality_positioning, skill_dribbling + movement_acceleration, skill_dribbling + mentality_positioning, skill_dribbling + mentality_penalties, movement_sprint_speed + mentality_aggression, mentality_aggression + mentality_positioning, mentality_positioning + mentality_penalties, 2*attacking_finishing, 2*attacking_volleys, 2*mentality_aggression, 2*mentality_positioning
- CB: attacking_crossing + attacking_heading_accuracy, attacking_crossing + mentality_interceptions, attacking_crossing + defending_